<!-- vscode-jupyter-toc -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->
<a id='toc0_'></a>**Содержание**    
- [Перегрузка операторов](#toc1_)    
  - [Основные операторы](#toc1_1_)    
  - [Перегрузка операторов](#toc1_2_)    
  - [Перегрузка операторов внутри классов](#toc1_3_)    
  - [Перегрузка инкремента и декремента](#toc1_4_)    
  - [Переопределение операторов ввода-вывода](#toc1_5_)    
  - [Умный указатель (общий взгляд)](#toc1_6_)    
  - [Оператор приведения типа](#toc1_7_)    
  - [Операторы с особым порядком вычисления](#toc1_8_)    
- [Правила (хорошего стиля) переопределения операторов](#toc2_)    
  - [Переопределение арифметических и битовых операторов](#toc2_1_)    
  - [“Правильное” переопределение операторов сравнения](#toc2_2_)    
  - [О чём стоит помнить](#toc2_3_)    
- [Умный указатель](#toc3_)    
- [Очень умный указатель](#toc4_)    
  - [А есть еще intrusive (invasive) pointers](#toc4_1_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- /vscode-jupyter-toc -->

# <a id='toc1_'></a>[Перегрузка операторов](#toc0_)

## <a id='toc1_1_'></a>[Основные операторы](#toc0_)

Арифметические
    
    ∙Унарные: префиксные + - ++ --, постфиксные ++ --
    ∙Бинарные:+ - * / % += -= *= /= %=
    
Битовые

    ∙Унарные:~ (побитовое отрицание 0101 -> 1010).
    ∙Бинарные:&(битовый and) |(битовый or) ^(битовый xor) &= |= ^= >> << (битовые сдвиги).

Логические

    ∙Унарные:!.
    ∙Бинарные:&& || (логического xor нет, т.к. он тут эквивалентен сравнению !=, т.к. "бит" просто один)
    ∙Сравнения:== != > < >= <=

Другие операторы

1. Оператор присваивания:=  
2. Специальные:

        ∙префиксные * (разыменование) & (взятие адреса),  
        ∙постфиксные -> (доступ к членам структуры по указателю) ->\* (a->\*b член, на который указывает b в объекте, на который указывает a),  
        ∙особые , (оператор последовательного выполнения) .(доступ к членам структуры) () :: (доступ к простанству имен)
    
3. Скобки: \[] (взятие по индексу и т.п. типа функции 1 аргумента) \() (типа аргументов функции) 
4. Оператор приведения(type)  
5. Тернарный оператор:x ? y : z  
6. Работа с памятью: new new[] delete delete[]


*) Пример оператора ',' b = (a +=c, a + d) - обеспечивает, что сначала a+=c, потом a+d  
\*\*) Обращение к члену структуры («член b объекта, на который указывает a») a->b (др. название селектор переменных класса)
\*\*\*) Обращение к члену структуры («член b объекта a») a.b (др. название селектор переменных класса) 

NB: оператор приведения A(B) - приводит типа B к А. Приведение типа это всегда конструктор типа (часто констр.по умолчанию). Зачем переопределять? Если тип B во внешней библиотеке, то мы не можем определить в нем конструктор с типом A. Т.е. в одну сторону задается конструктором, в другую оператором приведения. Че за чушь? 
 
Нельзя перегружать операторы . :: и тернарный оператор.

## <a id='toc1_2_'></a>[Перегрузка операторов](#toc0_)

На примере внешних функций:

    Vector  operator -( Vector  const& v) {
        return  Vector(-v.x, -v.y);                             // новый объект вектор с инициацией -х -у
    }
    
    Vector  operator +( Vector  const& v,Vector  const& w) {
        return  Vector(v.x + w.x, v.y + w.y);
    }
    
    Vector  operator *( Vector  const& v, double d) {
        return  Vector(v.x * d, v.y * d);
    }
    
    Vector  operator *( double d, Vector  const& v) {
        return v * d;                                           // тоже новый объект, т.к.умножение на число уже определено выше
    }

## <a id='toc1_3_'></a>[Перегрузка операторов внутри классов](#toc0_)

NB: Обязательно для (type) [] () -> ->* = (перегрузка только внутри класса)

    struct  Vector {
        Vector  operator -()  const { return  Vector(-x, -y); }     // унарный вариант (тут аргумент this)
        Vector  operator -( Vector  const& p) const {               // тут два аргумента this и внешений
            return  Vector(x - p.x, y - p.y);                       // this - второй аргумент
        }
        Vector & operator *=( double d) {
            x *= d; 
            y *= d;
            return *this;
        }
        double  operator [] (size_t i) const {
            return (i == 0) ? x : y;                                // вот такой вариант [i] (в стиле первый или другой)
        }
        bool     operator ()( double d)    const    { ... }
        void     operator ()( double a, double b)   { ... }         // для разных типов разное поведение оператора () - позволяет вести себя объектам как вызовам функций
        double x, y;
    };

\*) определение для одного типа перегрузки оператора и в виде внешней функции и в виде метода будет ошибка компиляции
\*\*) оператор * на число перегрузить внутри класса не получится из-за this

## <a id='toc1_4_'></a>[Перегрузка инкремента и декремента](#toc0_)

Для них есть специальных синтаксис перегрузки: 

    struct  BigNum {
        BigNum & operator ++() {    //prefix
            //increment
            ...
            return *this;
        }
        BigNum  operator ++(int) {  // postfix - чтобы отличать по синтаксису от ++x, используется dummy аргумент int
            BigNum  tmp(*this);     // сохраняем значение до инкремента 
            ++(* this);             // увеличиваем его
            return  tmp;            // а возвращаем значение до увеличения
        }                           // за счет этого, а также потому что возвращаем не по ссылке как выше, а копируем по значения
        ...                         // поэтому такая реализация и дольше (особенно если это не просто число, а сложный объект)
    };

## <a id='toc1_5_'></a>[Переопределение операторов ввода-вывода](#toc0_)

    #include  <iostream >   // в нем также подключается <iosfwd>, где определены cout cin
    
    struct  Vector { ... };
    
    std::istream& operator >>(std:: istream & is ,      // поток ввода 
                              Vector & p) {             // считываемый тип
        is >> p.x >> p.y;                               // чтобы работали последывательные считывания
        return  is;                                     // нужно возвращать ссылку на поток вывода (для следующего значения)
    }
    std:: ostream& operator <<(std:: ostream &os ,
                               Vector  const& p) {
        os << p.x << ’ ’ <<   p.y;                      // аналогично
        return  os;                                     // для последовательных выводов необходимо возвращать os
    }

*) переопределение и перегрузка тут используются как синонимы (это не очень хорошо, но в данном случае нет неоднозначности, т.к. виртуальные методы отсутствуют)

## <a id='toc1_6_'></a>[Умный указатель (общий взгляд)](#toc0_)

Оператор -> возвращает указатель
Реализует принцип: “Получение ресурса есть инициализация” Resource Acquisition Is Initialization (RAII)

    struct  SmartPtr {                                  // пример перегрузки оператора -> по типу 'умный указатель'
        Data & operator *()   const {return *data_ ;}   // чтоб он соблюдал интерфейс обычного указателя, он должен перегрузить
        Data * operator ->()  const {return  data_ ;}   // эти операции, причем по * должен возвращать тип указателя *
        Data * get()          const {return  data_ ;}
        ...
    private:
        Data * data_;
    };
    
    bool  operator == ( SmartPtr  const& p1,
                        SmartPtr  const& p2) {
        return  p1.get() == p2.get();
    }

## <a id='toc1_7_'></a>[Оператор приведения типа](#toc0_)
    
Применять нужно, когда на аргумент мы не может влиять (например создать дополнительный конструктор с приведением к нужному типу)  
Требование - реализация только в виде метода.  

    struct  String {                    
        operator  bool() const {                // к типу bool
            return  size_ != 0;
        }
        operator  char  const  *()  const {     // к типу строка (если пустая, то "")
            if (*this)
                return  data_;
            return "";
        }
    private:
        char * data_;
        size_t  size_;
    };

\*) в последнем стандарте 11 года разрешено использовать explicit для избежания неявных преобразований типа при использовании в момент инициализации (как в конструкторах от 1 параметра (или больше параметров, но 1й параметр не имеет значения по умолчанию))

## <a id='toc1_8_'></a>[Операторы с особым порядком вычисления](#toc0_)

В С++ таких перегружаемых операторов 3 штуки (&&, ||, ','). Их перегрузка имеет существенные особенности.

    int  main() {
        int a = 0;
        int b = 5;
        (a != 0) && (b = b / a);    // (bool) && (int) int преобразуется = (i==0) ? 0 : 1
                                    // причем если первый операнд true, второй вообще не вычисляется
                                    // это называется ленивое вычисление логики, short circuit logic
                                    // за счет этого мы защитились от деления на 0
        (a == 0) || (b = b / a);    // аналогично тут, второй операнд не вычисляется
        
        foo() && bar();             // второй оператор не будут вызываться, если первый false
        foo() || bar();
        foo(), bar();               // пример использования оператора последовательного вычисления
                                    // но вообще это малоизвестный и редкоиспользуемый оператор
                                    // (можно микроцикл так прописать и т.п. констркуции)
        a = foo(), bar();           // a присвоится возврат из bar()
    }
    
    // no lazy semantics
    Tribool  operator &&( Tribool  const& b1,   // следствием перегрузки станет в любом случае потеря ленивой логики
                          Tribool  const& b2) { // т.к. аргументы гарантированно вычисляются (при этом не гарантируется порядок)
        ...
    }

# <a id='toc2_'></a>[Правила (хорошего стиля) переопределения операторов](#toc0_)

## <a id='toc2_1_'></a>[Переопределение арифметических и битовых операторов](#toc0_)

Короткую версию оператора - в виде внешней функции, длинную - в классе.

    struct  String {
        String( char  const * cstr ) { ... }
        
        String & operator +=( String  const& s) {           // внутри, т.к. меняем левый аргумент
            ...                                             // удобно, т.к. удобнее получать доступ к нужным полям
            return *this;
        }
        
        //String operator+(String const& s2) const {...}    // а в таком виде реализации, из-за this не будет происходить автоматическое приведение типа 
                                                            // левого аргумента конструктором, т.е. случаи, когда оператор используется с конструктором,
                                                            // таким вариантом покрываются не полностью
    };
    
    // снаружи
    String  operator +( String s1, String  const& s2) {     // короткая версия + 
        return  s1 += s2;                                   // через длинную версию оператора +=
    }
    
    String  s1("world");
    String  s2 = "Hello " + s1;                             // приведение от C-строки к string в функции перегрузки оператора
                                                            // а если бы + был методом, то работал бы только вариант  s1 + "Hello "

## <a id='toc2_2_'></a>[“Правильное” переопределение операторов сравнения](#toc0_)

Правильный подход - в реализации сравнений через одну (максимум две) операции, а остальные - через них:


    bool  operator == ( String  const& a, String  const& b) {   // !(a<b)&&!(b<a) - самый простой вариант ==, при котором надо реализовать
        return  ...                                             // толоько ко оператор <. Однако, тут 2 вызова сравнения, что может быть неэффективным, 
    }                                                           // поэтому тут также хорошо бы дать определение сравнения по смыслу
    
    bool  operator !=( String  const& a, String  const& b) {    // тут через ==
        return  !(a == b);
    }
    bool  operator <( String  const& a, String  const& b) {     // ЕДИНСТВЕНННЫЙ ОПЕРАТОР который необходимо определять по смыслу
        return  ...
    }
    bool  operator >( String  const& a, String  const& b) {     // тут и далее через <
        return b < a;
    }
    bool  operator <=( String  const& a, String  const& b) {
        return  !(b < a);
    }
    bool  operator >=( String  const& a, String  const& b) {
        return  !(a < b);
    }

## <a id='toc2_3_'></a>[О чём стоит помнить](#toc0_)

∙Стоит соблюдать стандартную семантика операторов (сохранять общий смысл сравнения/сложения и т.д., если конечно не стоит задача запутать код или реализовать какой-то специальный тип операции).
    
    void  operator +(A const & a, A const& b) {}

∙Приоритет перегуженных операторов сохраняется таким же, как и у стандартных (использовать скобки, если сомнения).

    Vector a, b, c;
    c = a + a ^ b * a; //????? в случае вектора, это х.з. что получится в зависимости от порядка
    
∙Хотя бы один из параметров должен быть пользовательским. Иначе незачем перегружать операторы, в стандартных типах все и так ОК.

    void  operator *( double d, int i) {}

Отдельный вопрос - перегрузка new, delete, которые мы не рассматривали.

# <a id='toc3_'></a>[Умный указатель](#toc0_)

На предыдущих неделях вы уже набили достаточно шишек на ручном управлении памятью. Пришло время задуматься о более разумном подходе. Начнем с довольно простого случая — рассмотрим динамические объекты, время жизни которых ограничено блоком ({} — ограничивают блок в C++). Указатель на такой динамический объект внутри блока можно "связать" со временем жизни объекта на стеке: достаточно сохранить этот указатель в некотором объекте на стеке, а в деструкторе этого объекта вызвать delete. Так как деструктор объекта, созданного на стеке, вызывается автоматически при выходе из блока, то delete тоже будет вызван автоматически.

Такой класс-обертку мы будем называть ScopedPtr. Стоит заметить, что копирование такого объекта может приводить к серьезным проблемам, например, к повторному освобождению памяти (два объекта хранят внутри один и тот же указатель и вызов delete будет сделан дважды). Поэтому нужно запретить вызов конструктора копирования и оператора присваивания таких объектов. Добиться этого можно объявив их в private секции класса. При этом даже не нужно их реализовывать — снаружи класса никто не сможет их вызвать, а внутри класса мы этого делать не будем.

Какой интерфейс может быть у такого класса ScopedPtr? Кроме уже известных вам операторов * и ->, деструктора и конструктора, полезными могут оказаться следующие методы:

    get — возвращает указатель, сохраненный внутри ScopedPtr (например, чтобы передать его в какую-то функцию);
    release — забирает указатель у ScopedPtr и возвращает значение этого указателя, после вызова release ScopedPtr не должен освобождать память (например, чтобы вернуть этот указатель из функции);
    reset — метод заставляет ScopedPtr освободить старый указатель, а вместо него захватить новый (например, чтобы переиспользовать ScopedPtr, так как оператор присваивания запрещен).



# <a id='toc4_'></a>[Очень умный указатель](#toc0_)



Для ScopedPtr мы запретили копирование, однако, копирование можно и разрешить. Это позволит реализовать более продвинутый умный указатель — SharedPtr. Он отличается от ScopedPtr тем, что кроме хранения указателя на объект, он хранит еще и счетчик ссылок (количество объектов SharedPtr, которые хранят один и тот же указатель).

Имея такой счетчик, мы можем определить момент, когда на объект, выделенный в куче, не останется больше ссылок (когда счетчик ссылок станет равным 0), и освободить память.

Поддержка счетчика ссылок состоит из нескольких частей:

    в конструкторе SharedPtr от ненулевого указателя мы инициализируем счетчик ссылок в 1 (конструктор создает первый SharedPtr, который хранит указатель),
    в конструкторе копирования мы увеличиваем счетчик ссылок на 1, если копируемый SharedPtr содержит ненулевой указатель (конструктор копирования создает еще 
    один SharedPtr с указателем на тот же самый объект),
    в деструкторе мы уменьшаем значение счетчика на 1, если в объекте SharedPtr хранится ненулевой указатель (мы удаляем один SharedPtr, 
    который указывает на объект в куче),
    оператор присваивания уменьшает счетчик ссылок левого операнда на 1, если внутри левого SharedPtr хранится ненулевой указатель, 
    увеличивает счетчик правого SharedPtr на 1, если в правом SharedPtr хранится ненулевой указатель (обычное дело для оператора присваивания — 
    сначала освобождаем старые ресурсы, потом выделяем новые, но при этом нужно быть особенно внимательным с присваиванием самому себе).

Для класса SharedPtr могут оказаться полезными следующие методы (кроме операторов * и ->, конструктора копирования, оператора присваивания, деструктора и конструктора):

    метод get, как и в случае со ScopedPtr,
    метод reset — аналогичен reset у ScopedPtr, но освобождает память, только если счетчик ссылок после декремента равен 0.



## <a id='toc4_1_'></a>[А есть еще intrusive (invasive) pointers](#toc0_)

но это уже совсем другая история...